In [1]:
%load_ext autoreload
%autoreload 2
!hostname
!pwd
import os, sys
print(sys.executable)

fish-gcp005
/home/akarshkumar0101/nca-alife
/home/akarshkumar0101/miniconda3/envs/nca-alife-torch/bin/python


In [2]:
import os, sys, glob, pickle
from functools import partial

import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns
# sns.set_theme()

from tqdm.auto import tqdm
from einops import rearrange, reduce, repeat

In [3]:
# import experiment_utils
import time
import subprocess

# Launch Experiment (on this node)

In [4]:
header = '\n'.join(['#!/bin/bash', 'source ~/.virtualenvs/nca-alife-torch/bin/activate', 'cd ~/nca-alife/src'])
gpus = [4, 5, 6, 7]
log_dir = os.path.expanduser(f'~/nca-alife-data/logs/{time.strftime("%Y_%m_%d_%H_%M_%S")}')
exp_sh = f'./src/experiments/main.sh'
with open(exp_sh, 'r') as f:
    commands = [c for c in f.read().split('\n') if c]
print(f"{exp_sh} has {len(commands)} commands: ")
print('\n'.join(commands[:3]), '\n...')
print(f"Logging to {log_dir}")

./src/experiments/main.sh has 4 commands: 
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_multi_prompt_clip/prompt=0_seed=0" --img_size=64 --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --padding_mode="zeros" --dt=0.01 --p_drop=0.0 --rollout_steps=64 --bptt_steps=16 --prompt="a red apple;a green apple;a blue apple;a red apple"                                                     --n_augs=1 --augs="crop+pers" --bs=8 --lr=0.0005 --n_iters=1500000 --clip_grad_norm=1.0
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_multi_prompt_clip/prompt=1_seed=0" --img_size=64 --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --padding_mode="zeros" --dt=0.01 --p_drop=0.0 --rollout_steps=64 --bptt_steps=16 --prompt="a skinny cat;a normal cat;a fat cat;a very very fat cat"                                                --n_augs=1 --augs="crop+pers" --bs=8 --lr=0.0005 --n_iters=1500000 --clip_g

In [5]:
print(log_dir)

/home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55


In [6]:
!mkdir -p {log_dir}
!rm {os.path.dirname(log_dir)+'/latest'}
!ln -sf {log_dir} {os.path.dirname(log_dir)+'/latest'}
for i_gpu, gpu in enumerate(gpus):
    job_ids = np.arange(len(commands))[i_gpu::len(gpus)]
    cvd = f'export CUDA_VISIBLE_DEVICES={gpu}'
    txt = f"{header}\n{cvd}\n\n"
    for job_id in job_ids:
        txt += f"touch {log_dir}/job_{job_id:05d}.status\n"
        txt += f"{commands[job_id]} &> {log_dir}/job_{job_id:05d}.out\n"
        txt += f"echo $? > {log_dir}/job_{job_id:05d}.status\n\n"
    with open(f"{log_dir}/gpu_{gpu}.sh", "w") as f:
        f.write(txt)

In [8]:
print('\n'.join(txt.split('\n')[:12]))

#!/bin/bash
source ~/.virtualenvs/nca-alife-torch/bin/activate
cd ~/nca-alife/src
export CUDA_VISIBLE_DEVICES=7

touch /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/job_00003.status
python nca_multi_prompt_clip.py --seed=0 --save_dir="/home/akarshkumar0101/nca-alife-data/nca_multi_prompt_clip/prompt=3_seed=0" --img_size=64 --n_layers=2 --d_state=16 --d_embd=32 --init_state="point" --padding_mode="zeros" --dt=0.01 --p_drop=0.0 --rollout_steps=64 --bptt_steps=16 --prompt="random noise;self-replicating molecules;cells floating in primordial soup;alien animals roaming around" --n_augs=1 --augs="crop+pers" --bs=8 --lr=0.0005 --n_iters=1500000 --clip_grad_norm=1.0 &> /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/job_00003.out
echo $? > /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/job_00003.status




In [11]:
for i_gpu, gpu in enumerate(gpus):
    command = f"bash {log_dir}/gpu_{gpu}.sh"
    print(command)
    process = subprocess.Popen(command, shell=True)

bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/gpu_4.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/gpu_5.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/gpu_6.sh
bash /home/akarshkumar0101/nca-alife-data/logs/2024_06_21_22_39_55/gpu_7.sh


# Track Experiment

In [12]:
a = []
for job_id in range(len(commands)):
    try:
        with open(f'{log_dir}/job_{job_id:05d}.status', 'r') as f:
            a.append(int(f.read().strip()))
    except Exception as e:
        print(job_id, e)
        a.append(-1)
a = np.array(a)

0 invalid literal for int() with base 10: ''
1 invalid literal for int() with base 10: ''
2 invalid literal for int() with base 10: ''
3 invalid literal for int() with base 10: ''


In [13]:
a

array([-1, -1, -1, -1])

In [14]:
np.array(a).mean()

-1.0

In [15]:
len(commands)

4

In [29]:
for job_id in range(len(commands)):
    print(job_id)
    try:
        with open(f"{log_dir}/job_{job_id:05d}.out", "r") as f:
            print(f.readlines()[-1].strip())
    except Exception as e:
        print(e)

0
41%|████      | 613609/1500000 [5:26:43<7:44:10, 31.83it/s, loss=-0.396]
1
38%|███▊      | 569481/1500000 [5:26:43<8:51:58, 29.15it/s, loss=-0.373]
2
41%|████      | 616150/1500000 [5:26:43<7:44:23, 31.72it/s, loss=-0.332]
3
38%|███▊      | 572816/1500000 [5:26:43<8:46:48, 29.33it/s, loss=-0.334]
